In [1]:
import numpy as np
import pandas as pd
import dask.dataframe as ddf

columns = ['x1', 'x2', 'x3', 'y']
df = pd.DataFrame(np.random.normal(0, 1, (1_000_000, len(columns))).astype(np.float32), columns=columns)
random_data = ddf.from_pandas(df, npartitions=10)
random_data

/usr/local/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,x1,x2,x3,y
npartitions=10,,,,
0,float32,float32,float32,float32
100000,...,...,...,...
...,...,...,...,...
900000,...,...,...,...
999999,...,...,...,...


In [2]:
from feather_io import FeatherWriter
MiB = int(2**20)
fw = FeatherWriter("random_data/", features=columns[:-1], labels=columns[-1:])
partition_lengths = (
    random_data
    .repartition(partition_size=5*MiB)
    .map_partitions(fw)
    .compute()
)
partition_lengths

2022-12-08 10:59:11.607334: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


0    300000
1    300000
2    300000
3    100000
dtype: int64

In [3]:
from feather_io import FeatherReader
reader = FeatherReader("random_data/")
loaded = reader.as_dask_dataframe()
loaded

,x1,x2,x3,y
npartitions=5,,,,
,float32,float32,float32,float32
,...,...,...,...
...,...,...,...,...
,...,...,...,...
,...,...,...,...


In [4]:
reader.as_tf_dataset()

2022-12-08 12:20:37.102447: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<_UnbatchDataset element_spec=(TensorSpec(shape=(3,), dtype=tf.float32, name=None), TensorSpec(shape=(1,), dtype=tf.float32, name=None))>